In [33]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time as t
import requests
import csv
from selenium import webdriver
from pprint import pprint
from scraper import scrape_site
from pathlib import Path
import pandas as pd
from config import subscription_key
from urllib.error import HTTPError
from googleapi import google
from urllib.parse import urlencode, urlparse, parse_qs, urlsplit

import requests.exceptions
from collections import deque

from lxml.html import fromstring
from requests import get

In [34]:
file_to_output = Path("Data_Scraped/StL_Contacts_Final.csv")

In [35]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', headless=False, **executable_path)

In [36]:
url = "https://www.bizjournals.com/stlouis/potm?l=&time=&type=&ind=17"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Final_List = []
dict_list =[]
links = []

In [39]:

for link in soup.find_all("a", class_="item", href=True):
        links.append(link.get('href'))
#Get Next button
for link in soup.find_all("a", class_="btn btn--primary pull-right", href=True):
    button = link.get('href')


for i in links:
    if '/stlouis/potmsearch/detail/submission' in i:
        d = {}
        link = "https://www.bizjournals.com/" + i
        newI = i.split("?")
        slicefront = (newI[0])
        name = slicefront[46:]
        final_name = name.replace('_', " ")
        d["Name"] = final_name
        d["Link"] = link
        if d not in dict_list:
            dict_list.append(d)
        else:
            pass
    else:
        pass

In [40]:

try:
#     browser.click_link_by_partial_href('Next')
    browser.click_link_by_href(button) 
#     browser.find_link_by_text('Next').first.click()
#     browser.links.find_by_partial_text("Next").click()
    print("Went to the next page")
    html = ''
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
except:
    print("Finished all pages")


Finished all pages


In [41]:
len(dict_list)

34

In [42]:
pprint(dict_list)

[{'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6490961/Stephen_Davis?l=&time=&ind=17&type=&ro=0',
  'Name': 'Stephen Davis'},
 {'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6490956/Lindsay_Leible_Combs?l=&time=&ind=17&type=&ro=1',
  'Name': 'Lindsay Leible Combs'},
 {'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6489464/Brendan_McDermott?l=&time=&ind=17&type=&ro=2',
  'Name': 'Brendan McDermott'},
 {'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6489461/Peter_LeBlanc_Jr?l=&time=&ind=17&type=&ro=3',
  'Name': 'Peter LeBlanc Jr'},
 {'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6485546/Denise_Bloch?l=&time=&ind=17&type=&ro=4',
  'Name': 'Denise Bloch'},
 {'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6483512/Julia_M_Lang?l=&time=&ind=17&type=&ro=5',
  'Name': 'Julia M Lang'},
 {'Link': 'https://www.bizjournals.com//

In [43]:
# browser.visit("https://www.bizjournals.com/houston/potmsearch/detail/submission/6488386/Charles_M_Rosson?time=most_recent&ind=86&type=all&id=&ro=1")
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')
# date_info= soup.find_all('span', class_='meta-item')
# type_info= soup.find_all('a', class_='meta-item')
# info_list = soup.find_all('p', class_ = 'cXenseParse')
# education = soup.find_all("span", class_ = "cXenseParse")
# # print(education[0].text)
# # print(type_info[0].text)
# # print(type_info[1].text)
# print(date_info[4].text)

In [44]:
for i in dict_list:

        try:
                browser.visit(i["Link"])
                t.sleep(1)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                info_list = soup.find_all('p', class_ = 'cXenseParse')
                date_info = soup.find_all('span', class_='meta-item')
                type_info= soup.find_all('a', class_='meta-item')
                education = soup.find_all("span", class_ = "cXenseParse")
                try: 
                    i["Title"] = info_list[0].text
                    i["Blurb"] = info_list[1].text
                    i["Publish Date"] = date_info[4].text
                    i["New Hire/Promotion"] = type_info[0].text
                    i["Industry"] = type_info[1].text
                    i["Education"] = education[0].text
                except:
                    print(f"Missing info for {i['Name']}")
                Final_List.append(i)

        except HTTPError as error:
            print(error)
length_Final_List = len(Final_List)
print(f"Finished pulling. {length_Final_List} items pulled")

Missing info for Cynthia Garnholz
Missing info for Thomas Addis
Missing info for Cynthia Garnholz
Missing info for Kristen Sparks
Missing info for David Deterding
Missing info for W DUDLEY MCCARTER
Finished pulling. 34 items pulled


In [45]:
len(Final_List)

34

In [46]:
pprint(Final_List)

[{'Blurb': 'Carmody MacDonald P.C. is proud to announce that Stephen Davis has '
           'been promoted to partner. Stephen concentrates his practice in '
           'real estate and commercial litigation. He advises and counsels '
           'clients on real estate and corporate governance matters, including '
           'commercial real estate deals, and represents over 200 homeowner '
           'and condominium associations in the St. Louis region. He lives in '
           'University City with his wife, Julie, and their daughter, Arielle.',
  'Education': 'EDUCATION:\xa0\xa0Washington University School of Law',
  'Industry': '\n'
              '                                        Legal '
              'Services                                    ',
  'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6490961/Stephen_Davis?l=&time=&ind=17&type=&ro=0',
  'Name': 'Stephen Davis',
  'New Hire/Promotion': '\nPromotion\n',
  'Publish Date': 'December 14, 2

In [47]:
assert subscription_key

In [48]:
for i in Final_List:
    try:
        search_url = "https://api.bing.microsoft.com/v7.0/search"
        search_term = i["Name"] + ' ' + i["Title"]
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
        print(f"Found {i['Name']}!")
    except:
        print(f"Could not find {i['Name']}")
        print(subscription_key)
        print(response)
    t.sleep(1)
print(f"Finished fetching Bing Searches.")

Found Stephen Davis!
Found Lindsay Leible Combs!
Found Brendan McDermott!
Found Peter LeBlanc Jr!
Found Denise Bloch!
Found Julia M Lang!
Found Cynthia Garnholz!
Found Dormie Ko!
Found Joe Palumbo!
Found Thomas Addis!
Found Thomas Hayek!
Found Richard Behr!
Found Cynthia Garnholz!
Found Kristen Sparks!
Found Michael Hibey!
Found Laura Fitzpatrick!
Found David Deterding!
Found Renee Cipriano!
Found Erick S Creach!
Found Stacey Murphy!
Found John Galvin!
Found Suzanne Galvin!
Found Colin Halpin!
Found Brooke Wheelwright!
Found Phillip A Casey!
Found Mitchell Martin!
Found W DUDLEY MCCARTER!
Found Laurence Nassif!
Found Amy Garrett!
Found Katie S Davies!
Found Julie Logan!
Found Cass J Swanson!
Found Erik Goltzer!
Found Bridget Halquist!
Finished fetching Bing Searches.


In [49]:
Final_List

[{'Name': 'Stephen Davis',
  'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6490961/Stephen_Davis?l=&time=&ind=17&type=&ro=0',
  'Title': 'Partner at Carmody MacDonald PC',
  'Blurb': 'Carmody MacDonald P.C. is proud to announce that Stephen Davis has been promoted to partner. Stephen concentrates his practice in real estate and commercial litigation. He advises and counsels clients on real estate and corporate governance matters, including commercial real estate deals, and represents over 200 homeowner and condominium associations in the St. Louis region. He lives in University City with his wife, Julie, and their daughter, Arielle.',
  'Publish Date': 'December 14, 2020',
  'New Hire/Promotion': '\nPromotion\n',
  'Industry': '\n                                        Legal Services                                    ',
  'Education': 'EDUCATION:\xa0\xa0Washington University School of Law',
  'Bing Search Link': 'https://www.carmodymacdonald.com/stephen-da

In [50]:
for i in Final_List:
    emails = set()
    try:
        response = requests.get(i["Bing Search Link"])
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
        emails.update(new_emails)
        i['Bing Emails'] = emails
        if emails == set():
            print(f'Set empty on {i["Name"]}')
            emails = set()
        else:
            print(f"Successfully captured an email for {i['Name']}")
            emails = set()
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors and continue with next url
        print(f"Could not find {i['Name']}")
        continue
print(f"Finished all email grabs")

Successfully captured an email for Stephen Davis
Successfully captured an email for Lindsay Leible Combs
Set empty on Brendan McDermott
Set empty on Peter LeBlanc Jr
Successfully captured an email for Denise Bloch
Successfully captured an email for Julia M Lang
Set empty on Cynthia Garnholz
Successfully captured an email for Dormie Ko
Successfully captured an email for Joe Palumbo
Set empty on Thomas Addis
Successfully captured an email for Thomas Hayek
Successfully captured an email for Richard Behr
Set empty on Cynthia Garnholz
Successfully captured an email for Kristen Sparks
Set empty on Michael Hibey
Set empty on Laura Fitzpatrick
Successfully captured an email for David Deterding
Successfully captured an email for Renee Cipriano
Successfully captured an email for Erick S Creach
Set empty on Stacey Murphy
Successfully captured an email for John Galvin
Successfully captured an email for Suzanne Galvin
Set empty on Colin Halpin
Set empty on Brooke Wheelwright
Set empty on Phillip A 

In [51]:
Final_List

[{'Name': 'Stephen Davis',
  'Link': 'https://www.bizjournals.com//stlouis/potmsearch/detail/submission/6490961/Stephen_Davis?l=&time=&ind=17&type=&ro=0',
  'Title': 'Partner at Carmody MacDonald PC',
  'Blurb': 'Carmody MacDonald P.C. is proud to announce that Stephen Davis has been promoted to partner. Stephen concentrates his practice in real estate and commercial litigation. He advises and counsels clients on real estate and corporate governance matters, including commercial real estate deals, and represents over 200 homeowner and condominium associations in the St. Louis region. He lives in University City with his wife, Julie, and their daughter, Arielle.',
  'Publish Date': 'December 14, 2020',
  'New Hire/Promotion': '\nPromotion\n',
  'Industry': '\n                                        Legal Services                                    ',
  'Education': 'EDUCATION:\xa0\xa0Washington University School of Law',
  'Bing Search Link': 'https://www.carmodymacdonald.com/stephen-da

In [52]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,Bing Search Link,Bing Emails
0,Stephen Davis,https://www.bizjournals.com//stlouis/potmsearc...,Partner at Carmody MacDonald PC,Carmody MacDonald P.C. is proud to announce th...,"December 14, 2020",\nPromotion\n,\n Lega...,EDUCATION: Washington University School of Law,https://www.carmodymacdonald.com/stephen-davis,{sgd@carmodymacdonald.com}
1,Lindsay Leible Combs,https://www.bizjournals.com//stlouis/potmsearc...,Partner at Carmody MacDonald PC,Carmody MacDonald P.C. is proud to announce th...,"December 14, 2020",\nPromotion\n,\n Lega...,EDUCATION: Saint Louis University School of Law,https://www.carmodymacdonald.com/lindsay-leible,{ltl@carmodymacdonald.com}
2,Brendan McDermott,https://www.bizjournals.com//stlouis/potmsearc...,Attorney at Lowenhaupt & Chasnoff LLC,Brendan McDermott has joined Lowenhaupt & Chas...,"October 29, 2020",\nNew Hire\n,\n Law ...,EDUCATION: Washington University School of Law,https://lowenchas.com/team/brendan-t-mcdermott/,{}
3,Peter LeBlanc Jr,https://www.bizjournals.com//stlouis/potmsearc...,Attorney at Lowenhaupt & Chasnoff LLC,"Peter LeBlanc, Jr. has joined Lowenhaupt & Cha...","October 29, 2020",\nNew Hire\n,\n Law ...,EDUCATION: Saint Louis University-Main Campus...,https://lowenchas.com/team/peter-j-leblanc-jr/,{}
4,Denise Bloch,https://www.bizjournals.com//stlouis/potmsearc...,"Of Counsel at Evans and Dixon, LLC","Evans & Dixon, LLC is pleased to announce that...","July 10, 2020",\nNew Hire\n,\n Lega...,EDUCATION: Saint Louis University School of Law,https://www.evans-dixon.com/bio/1323/Denise-H-...,"{contact@evans-dixon.com, dbloch@evans-dixon.com}"


In [53]:
FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')
FL["Industry"] = FL["Industry"].str.strip('\n')
FL["Blurb"] = FL["Blurb"].str.strip('\n')
FL["Blurb"] = FL["Blurb"].replace(r'\\n',' ', regex=True) 
FL["Blurb"] = FL["Blurb"].replace('\n','', regex=True)
FL["Blurb"] = FL["Blurb"].replace('\n\n',' ', regex=True)
FL["Industry"] = FL["Industry"].str.strip()
FL["Education"] = FL["Education"].str.strip('EDUCATION:  ')
FL["Education"] = FL["Education"].str.strip()
# Split the Name column into First and Last
new = FL["Name"].str.split(" ", n = 1, expand = True) 
# making separate first name column from new data frame 
FL["First Name"]= new[0]  
# making separate last name column from new data frame 
FL["Last Name"]= new[1]
# Split the Title column into New Title and Firm
spltTitle = FL["Title"].str.split(" at ", n = 1, expand = True) 
# making separate first name column from new data frame 
FL["New Title"]= spltTitle[0] 
# making separate last name column from new data frame 
FL["Firm"]= spltTitle[1]

In [54]:
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,Bing Search Link,Bing Emails,First Name,Last Name,New Title,Firm
0,Stephen Davis,https://www.bizjournals.com//stlouis/potmsearc...,Partner at Carmody MacDonald PC,Carmody MacDonald P.C. is proud to announce th...,"December 14, 2020",Promotion,Legal Services,Washington University School of Law,https://www.carmodymacdonald.com/stephen-davis,{sgd@carmodymacdonald.com},Stephen,Davis,Partner,Carmody MacDonald PC
1,Lindsay Leible Combs,https://www.bizjournals.com//stlouis/potmsearc...,Partner at Carmody MacDonald PC,Carmody MacDonald P.C. is proud to announce th...,"December 14, 2020",Promotion,Legal Services,Saint Louis University School of Law,https://www.carmodymacdonald.com/lindsay-leible,{ltl@carmodymacdonald.com},Lindsay,Leible Combs,Partner,Carmody MacDonald PC
2,Brendan McDermott,https://www.bizjournals.com//stlouis/potmsearc...,Attorney at Lowenhaupt & Chasnoff LLC,Brendan McDermott has joined Lowenhaupt & Chas...,"October 29, 2020",New Hire,Law Practice,Washington University School of Law,https://lowenchas.com/team/brendan-t-mcdermott/,{},Brendan,McDermott,Attorney,Lowenhaupt & Chasnoff LLC
3,Peter LeBlanc Jr,https://www.bizjournals.com//stlouis/potmsearc...,Attorney at Lowenhaupt & Chasnoff LLC,"Peter LeBlanc, Jr. has joined Lowenhaupt & Cha...","October 29, 2020",New Hire,Law Practice,Saint Louis University-Main Campus (Saint Loui...,https://lowenchas.com/team/peter-j-leblanc-jr/,{},Peter,LeBlanc Jr,Attorney,Lowenhaupt & Chasnoff LLC
4,Denise Bloch,https://www.bizjournals.com//stlouis/potmsearc...,"Of Counsel at Evans and Dixon, LLC","Evans & Dixon, LLC is pleased to announce that...","July 10, 2020",New Hire,Legal Services,Saint Louis University School of Law,https://www.evans-dixon.com/bio/1323/Denise-H-...,"{contact@evans-dixon.com, dbloch@evans-dixon.com}",Denise,Bloch,Of Counsel,"Evans and Dixon, LLC"
5,Julia M Lang,https://www.bizjournals.com//stlouis/potmsearc...,Associate at Carmody MacDonald PC,Julia M. Lang is an associate in Carmody MacDo...,"April 30, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,https://www.carmodymacdonald.com/julia-lang,{jml@carmodymacdonald.com},Julia,M Lang,Associate,Carmody MacDonald PC
6,Cynthia Garnholz,https://www.bizjournals.com//stlouis/potmsearc...,Owner/Attorney at Law at Garnholz | Sparks Fam...,"Cynthia Garnholz, JD, will receive a 2020 Miss...","April 27, 2020",Professional Recognition,Legal Services,NaN,https://www.google.com/,{},Cynthia,Garnholz,Owner/Attorney,Law at Garnholz | Sparks Family Lawyers
7,Dormie Ko,https://www.bizjournals.com//stlouis/potmsearc...,Associate Attorney at Carmody MacDonald PC,Carmody MacDonald P.C. welcomes Dormie Ko to t...,"April 20, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,https://www.carmodymacdonald.com/dormie-ko,{dko@carmodymacdonald.com},Dormie,Ko,Associate Attorney,Carmody MacDonald PC
8,Joe Palumbo,https://www.bizjournals.com//stlouis/potmsearc...,Associate Attorney at Carmody MacDonald PC,Joe Palumbo joined Carmody MacDonald P.C. as a...,"April 20, 2020",New Hire,Legal Services,"University of Missouri (Columbia, MO)",https://www.carmodymacdonald.com/joseph-palumbo,{jdp@carmodymacdonald.com},Joe,Palumbo,Associate Attorney,Carmody MacDonald PC
9,Thomas Addis,https://www.bizjournals.com//stlouis/potmsearc...,"Associate at Goldenberg Heller & Antognoli, P.C.",Tom Addis has joined the Commercial Practice G...,"March 16, 2020",New Hire,Legal Services,NaN,https://www.goldenbergheller.com/,{},Thomas,Addis,Associate,"Goldenberg Heller & Antognoli, P.C."


In [55]:
FL = FL.drop(columns =["Name"])

In [56]:
#Reorder columns
FL = FL[["First Name", "Last Name", "New Title", "Firm", "Publish Date", "New Hire/Promotion","Industry", "Education", "Blurb", "Link", "Bing Search Link", "Bing Emails"]]
FL

,First Name,Last Name,New Title,Firm,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link,Bing Search Link,Bing Emails
0,Stephen,Davis,Partner,Carmody MacDonald PC,"December 14, 2020",Promotion,Legal Services,Washington University School of Law,Carmody MacDonald P.C. is proud to announce th...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/stephen-davis,{sgd@carmodymacdonald.com}
1,Lindsay,Leible Combs,Partner,Carmody MacDonald PC,"December 14, 2020",Promotion,Legal Services,Saint Louis University School of Law,Carmody MacDonald P.C. is proud to announce th...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/lindsay-leible,{ltl@carmodymacdonald.com}
2,Brendan,McDermott,Attorney,Lowenhaupt & Chasnoff LLC,"October 29, 2020",New Hire,Law Practice,Washington University School of Law,Brendan McDermott has joined Lowenhaupt & Chas...,https://www.bizjournals.com//stlouis/potmsearc...,https://lowenchas.com/team/brendan-t-mcdermott/,{}
3,Peter,LeBlanc Jr,Attorney,Lowenhaupt & Chasnoff LLC,"October 29, 2020",New Hire,Law Practice,Saint Louis University-Main Campus (Saint Loui...,"Peter LeBlanc, Jr. has joined Lowenhaupt & Cha...",https://www.bizjournals.com//stlouis/potmsearc...,https://lowenchas.com/team/peter-j-leblanc-jr/,{}
4,Denise,Bloch,Of Counsel,"Evans and Dixon, LLC","July 10, 2020",New Hire,Legal Services,Saint Louis University School of Law,"Evans & Dixon, LLC is pleased to announce that...",https://www.bizjournals.com//stlouis/potmsearc...,https://www.evans-dixon.com/bio/1323/Denise-H-...,"{contact@evans-dixon.com, dbloch@evans-dixon.com}"
5,Julia,M Lang,Associate,Carmody MacDonald PC,"April 30, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,Julia M. Lang is an associate in Carmody MacDo...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/julia-lang,{jml@carmodymacdonald.com}
6,Cynthia,Garnholz,Owner/Attorney,Law at Garnholz | Sparks Family Lawyers,"April 27, 2020",Professional Recognition,Legal Services,NaN,"Cynthia Garnholz, JD, will receive a 2020 Miss...",https://www.bizjournals.com//stlouis/potmsearc...,https://www.google.com/,{}
7,Dormie,Ko,Associate Attorney,Carmody MacDonald PC,"April 20, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,Carmody MacDonald P.C. welcomes Dormie Ko to t...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/dormie-ko,{dko@carmodymacdonald.com}
8,Joe,Palumbo,Associate Attorney,Carmody MacDonald PC,"April 20, 2020",New Hire,Legal Services,"University of Missouri (Columbia, MO)",Joe Palumbo joined Carmody MacDonald P.C. as a...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/joseph-palumbo,{jdp@carmodymacdonald.com}
9,Thomas,Addis,Associate,"Goldenberg Heller & Antognoli, P.C.","March 16, 2020",New Hire,Legal Services,NaN,Tom Addis has joined the Commercial Practice G...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.goldenbergheller.com/,{}


In [57]:
pd.set_option('display.max_rows', None)
FL

,First Name,Last Name,New Title,Firm,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link,Bing Search Link,Bing Emails
0,Stephen,Davis,Partner,Carmody MacDonald PC,"December 14, 2020",Promotion,Legal Services,Washington University School of Law,Carmody MacDonald P.C. is proud to announce th...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/stephen-davis,{sgd@carmodymacdonald.com}
1,Lindsay,Leible Combs,Partner,Carmody MacDonald PC,"December 14, 2020",Promotion,Legal Services,Saint Louis University School of Law,Carmody MacDonald P.C. is proud to announce th...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/lindsay-leible,{ltl@carmodymacdonald.com}
2,Brendan,McDermott,Attorney,Lowenhaupt & Chasnoff LLC,"October 29, 2020",New Hire,Law Practice,Washington University School of Law,Brendan McDermott has joined Lowenhaupt & Chas...,https://www.bizjournals.com//stlouis/potmsearc...,https://lowenchas.com/team/brendan-t-mcdermott/,{}
3,Peter,LeBlanc Jr,Attorney,Lowenhaupt & Chasnoff LLC,"October 29, 2020",New Hire,Law Practice,Saint Louis University-Main Campus (Saint Loui...,"Peter LeBlanc, Jr. has joined Lowenhaupt & Cha...",https://www.bizjournals.com//stlouis/potmsearc...,https://lowenchas.com/team/peter-j-leblanc-jr/,{}
4,Denise,Bloch,Of Counsel,"Evans and Dixon, LLC","July 10, 2020",New Hire,Legal Services,Saint Louis University School of Law,"Evans & Dixon, LLC is pleased to announce that...",https://www.bizjournals.com//stlouis/potmsearc...,https://www.evans-dixon.com/bio/1323/Denise-H-...,"{contact@evans-dixon.com, dbloch@evans-dixon.com}"
5,Julia,M Lang,Associate,Carmody MacDonald PC,"April 30, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,Julia M. Lang is an associate in Carmody MacDo...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/julia-lang,{jml@carmodymacdonald.com}
6,Cynthia,Garnholz,Owner/Attorney,Law at Garnholz | Sparks Family Lawyers,"April 27, 2020",Professional Recognition,Legal Services,NaN,"Cynthia Garnholz, JD, will receive a 2020 Miss...",https://www.bizjournals.com//stlouis/potmsearc...,https://www.google.com/,{}
7,Dormie,Ko,Associate Attorney,Carmody MacDonald PC,"April 20, 2020",New Hire,Legal Services,Washington University in St Louis (Saint Louis...,Carmody MacDonald P.C. welcomes Dormie Ko to t...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/dormie-ko,{dko@carmodymacdonald.com}
8,Joe,Palumbo,Associate Attorney,Carmody MacDonald PC,"April 20, 2020",New Hire,Legal Services,"University of Missouri (Columbia, MO)",Joe Palumbo joined Carmody MacDonald P.C. as a...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.carmodymacdonald.com/joseph-palumbo,{jdp@carmodymacdonald.com}
9,Thomas,Addis,Associate,"Goldenberg Heller & Antognoli, P.C.","March 16, 2020",New Hire,Legal Services,NaN,Tom Addis has joined the Commercial Practice G...,https://www.bizjournals.com//stlouis/potmsearc...,https://www.goldenbergheller.com/,{}


In [58]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

## Searching Bing for website and emails

In [22]:
# for i in Final_List:
#     try:
#         search_url = "https://api.bing.microsoft.com/v7.0/search"
#         search_term = i["Name"] + ' ' + i["Title"]
#         headers = {"Ocp-Apim-Subscription-Key": subscription_key}
#         params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
#         response = requests.get(search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()
#         i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
#         print(f"Found {i['Name']}!")
#     except:
#         print(f"Could not find {i['Name']}")
#         print(subscription_key)
#         print(response)
#     t.sleep(1)

In [181]:
# for i in Final_List:
#     try:
#         search_url = "https://api.bing.microsoft.com/v7.0/search"
#         search_term = i["Name"] + ' ' + i["Title"]
#         headers = {"Ocp-Apim-Subscription-Key": subscription_key}
#         params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
#         response = requests.get(search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()
#         i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
#         print(f"Found {i['Name']}!")
#     except:
#         print(f"Could not find {i['Name']}")
#     t.sleep(1)

In [23]:
# Final_List

In [24]:
# FL = pd.DataFrame(Final_List)
# FL.head()

In [78]:
# FL.isna().sum()

In [25]:
# FL.Title.value_counts()

In [26]:
# Saving the dataframe to a csv
# df_to_csv = FL.to_csv(file_to_output)

In [ ]:
#Convert Bing Links to a list
# BSL = FL["Bing Search Link"].to_list()
# BSL

### Testing the email search

In [ ]:
# # starting url. replace google with your own url.
# starting_url = 'https://www.kslaw.com/people?locale=en&office_id=4'

# # a queue of urls to be crawled
# unprocessed_urls = deque([starting_url])

# # set of already crawled urls for email
# processed_urls = set()

# # a set of fetched emails
# emails = set()

# # process urls one by one from unprocessed_url queue until queue is empty
# while len(unprocessed_urls):

#     # move next url from the queue to the set of processed urls
#     url = unprocessed_urls.popleft()
#     processed_urls.add(url)

#     # extract base url to resolve relative links
#     parts = urlsplit(url)
#     base_url = "{0.scheme}://{0.netloc}".format(parts)
#     path = url[:url.rfind('/')+1] if '/' in parts.path else url

#     # get url's content
#     print("Crawling URL %s" % url)
#     try:
#         response = requests.get(url)
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         continue

#     # extract all email addresses and add them into the resulting set
#     # You may edit the regular expression as per your requirement
#     new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#     emails.update(new_emails)
#     print(emails)
#     # create a beutiful soup for the html document
#     soup = BeautifulSoup(response.text, 'lxml')
#     # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
#     for anchor in soup.find_all("a"):
#         # extract link url from the anchor
#         link = anchor.attrs["href"] if "href" in anchor.attrs else ''
#         # resolve relative links (starting with /)
#         if link.startswith('/'):
#             link = base_url + link
#         elif not link.startswith('http'):
#             link = path + link
#         # add the new url to the queue if it was not in unprocessed list nor in processed list yet
#         if not link in unprocessed_urls and not link in processed_urls:
#             unprocessed_urls.append(link)

In [79]:
# for i in Final_List:
# #     print(i["Bing Search Link"])
#     response = requests.get(i["Bing Search Link"])
#     print(response)

In [27]:
# for i in Final_List:
#     emails = set()
#     try:
#         response = requests.get(i["Bing Search Link"])
#         new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#         emails.update(new_emails)
#         i['Bing Emails'] = emails
#         if emails == set():
#             print(f'Set empty on {i["Name"]}')
#             emails = set()
#         else:
#             print(f"Successfully captured an email for {i['Name']}")
#             emails = set()
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         print(f"Could not find {i['Name']}")
#         continue
# print(f"Finished all email grabs")

In [28]:
# Final_List

In [80]:
# FL = pd.DataFrame(Final_List)
# FL.head()

In [69]:
# FL['Bing Emails'] = FL['Bing Emails'].astype(str)

In [72]:
# FL["Bing Emails"] = FL["Bing Emails"].str.strip('{}')
# FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')


In [81]:
# FL

In [74]:
# # Saving the dataframe to a csv
# df_to_csv = FL.to_csv(file_to_output)

In [ ]:
# Final_List

In [8]:
# edit_csv = pd.read_csv("Data_Scraped/Contacts_Final.csv")

In [29]:
# edit_csv.head()